In [1]:
import numpy as np
import pandas as pd

In [3]:
ipl=pd.read_csv("C:\\Users\\MEGH BAVARVA\\Desktop\\Data Scientist\\Project\\API_development\\IPL_Matches_2008_2022.csv")
ipl.head(2)

# team1 = ipl['Team1'].unique()
# team2 = ipl['Team2'].unique()

# all_teams = pd.unique(np.concatenate((team1, team2)))
# print(all_teams)

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,N,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon


WinningTeam
Mumbai Indians                 131
Chennai Super Kings            121
Kolkata Knight Riders          114
Royal Challengers Bangalore    109
Rajasthan Royals                96
Kings XI Punjab                 88
Sunrisers Hyderabad             75
Delhi Daredevils                67
Delhi Capitals                  36
Deccan Chargers                 29
Gujarat Lions                   13
Punjab Kings                    13
Gujarat Titans                  12
Pune Warriors                   12
Rising Pune Supergiant          10
Lucknow Super Giants             9
Kochi Tuskers Kerala             6
Rising Pune Supergiants          5
Name: count, dtype: int64

In [ ]:
def team_vs_team(team1, team2):
    match=ipl[((ipl['Team1']==team1) & (ipl['Team2']==team2))
          |((ipl['Team1']==team2) & (ipl['Team2']==team1))]

    no_of_match=match.shape[0]
    print("No of matches Played : ", no_of_match)

    team1_win=match[match['WinningTeam']==team1].shape[0]
    team2_win=match[match['WinningTeam']==team2].shape[0]
    
    draws=no_of_match-(team1_win+team2_win)
    
    print("No of draws: ", draws)
    print(team1, " win: ", team1_win)
    print(team2, " win: ", team2_win)

    team1_win_percentage=(team1_win/no_of_match)*100
    team2_win_percentage=(team2_win/no_of_match)*100

    print(team1, " winning percentage: ", team1_win_percentage)
    print(team2, " winning percentage: ", team2_win_percentage)
    
team_vs_team('Mumbai Indians', 'Chennai Super Kings')

No of matches Played :  34
No of draws:  0
Mumbai Indians  win:  20
Chennai Super Kings  win:  14
Mumbai Indians  winning percentage:  58.82352941176471
Chennai Super Kings  winning percentage:  41.17647058823529


In [6]:
url="https://docs.google.com/spreadsheets/d/e/2PACX-1vRy2DUdUbaKx_Co9F0FSnIlyS-8kp4aKv_I0-qzNeghiZHAI_hw94gKG22XTxNJHMFnFVKsO4xWOdIs/pub?gid=1655759976&single=true&output=csv"
ipl2=pd.read_csv(url)
change_team={
    'Delhi Daredevils':'Delhi Capitals',
    'Kings XI Punjab ':'Punjab Kings',
    'Rising Pune Supergiants ':'Rising Pune Supergiant'
}

ipl2.replace(change_team.keys(),change_team.values(),inplace=True) 

def winning_team():
    df = ipl2[~ipl2['WinningTeam'].isnull()]  # remove rows with null WinningTeam

    team_1 = df['Team1'].unique()
    team_2 = df['Team2'].unique()

    total_team = np.union1d(team_1, team_2)

    data_list = []

    for team in total_team:
        total_matches = df[(df['Team1'] == team) | (df['Team2'] == team)]
        total_wins = total_matches[total_matches['WinningTeam'] == team]

        # home and away
        home_wins = total_matches[(total_matches['WinningTeam'] == team) & (total_matches['Team1'] == team)]
        away_wins = total_matches[(total_matches['WinningTeam'] == team) & (total_matches['Team2'] == team)]

        # calculate percentages
        wins_perc = (total_wins.shape[0] / total_matches.shape[0]) * 100 if total_matches.shape[0] > 0 else 0
        home_wins_perc = (home_wins.shape[0] / total_wins.shape[0]) * 100 if total_wins.shape[0] > 0 else 0
        away_wins_perc = (away_wins.shape[0] / total_wins.shape[0]) * 100 if total_wins.shape[0] > 0 else 0

        # collect data
        data_list.append([team, total_matches.shape[0], round(wins_perc, 2),
                          round(home_wins_perc, 2), round(away_wins_perc, 2)])

    result_df = pd.DataFrame(data_list, columns=["Team", "Matches_Played", "Win_Percentage", "Home_Wins_Percentage", "Away_Wins_Percentage"])
    result_df = result_df.sort_values("Win_Percentage", ascending=False).set_index("Team")


    return result_df


# winning_team()

# ipl2.head(2)

In [53]:
def matched_played(df, team):
    return df[(df['Team1'] == team) | (df['Team2'] == team)].shape[0]

def match_won(df, team):
    return df[df['WinningTeam'] == team].shape[0]

def no_result(df, team):
    return df[((df['Team1'] == team) | (df['Team2'] == team)) & (df['WinningTeam'].isnull())].shape[0]

def point_table(season):
    df = ipl2[ipl2['Season'].astype(str) == str(season)]  # Ensure comparison is string-based
    teams = np.union1d(df['Team1'].unique(), df['Team2'].unique())

    new_dataframe = pd.DataFrame()
    new_dataframe['Team_name'] = teams

    new_dataframe['Match_played'] = new_dataframe['Team_name'].apply(lambda x: matched_played(df, x))
    new_dataframe['Wins'] = new_dataframe['Team_name'].apply(lambda x: match_won(df, x))
    new_dataframe['No_result'] = new_dataframe['Team_name'].apply(lambda x: no_result(df, x))
    new_dataframe['Losses'] = new_dataframe['Match_played'] - new_dataframe['Wins']-new_dataframe['No_result']

    new_dataframe['Points'] = new_dataframe['Wins'] * 2 + new_dataframe['No_result']
    
    new_dataframe.sort_values('Points', ascending=False, inplace=True)
    new_dataframe.set_index('Team_name', inplace=True)

    return new_dataframe


point_table(2016)

,Match_played,Wins,No_result,Losses,Points
Team_name,,,,,
Sunrisers Hyderabad,17,11,0,6,22
Gujarat Lions,16,9,0,7,18
Royal Challengers Bangalore,16,9,0,7,18
Kolkata Knight Riders,15,8,0,7,16
Mumbai Indians,14,7,0,7,14
Delhi Capitals,14,7,0,7,14
Rising Pune Supergiants,14,5,0,9,10
Kings XI Punjab,14,4,0,10,8


In [67]:
def point_table_extension(season):
    season = str(season)
    new_df = point_table(season)
    new_df['SeasonPosition'] = np.nan  # Add this line to avoid assignment errors

    # Final
    final = ipl2[(ipl2['Season'] == season) & (ipl2['MatchNumber'] == 'Final')]
    if not final.empty:
        if final['WinningTeam'].values[0] == final['Team1'].values[0]:
            new_df.loc[final['Team1'].values[0], 'SeasonPosition'] = 'Winner'
            new_df.loc[final['Team2'].values[0], 'SeasonPosition'] = 'Runner Up'
        else:
            new_df.loc[final['Team2'].values[0], 'SeasonPosition'] = 'Winner'
            new_df.loc[final['Team1'].values[0], 'SeasonPosition'] = 'Runner Up'

    # Qualifier 2
    qualifier = ipl2[(ipl2['Season'] == season) & (ipl2['MatchNumber'] == 'Qualifier 2')]
    if not qualifier.empty:
        if qualifier['WinningTeam'].values[0] == qualifier['Team1'].values[0]:
            new_df.loc[qualifier['Team2'].values[0], 'SeasonPosition'] = '3rd Place'
        else:
            new_df.loc[qualifier['Team1'].values[0], 'SeasonPosition'] = '3rd Place'

    # Eliminator
    eliminator = ipl2[(ipl2['Season'] == season) & (ipl2['MatchNumber'] == 'Eliminator')]
    if not eliminator.empty:
        if eliminator['WinningTeam'].values[0] == eliminator['Team1'].values[0]:
            new_df.loc[eliminator['Team2'].values[0], 'SeasonPosition'] = '4th Place'
        else:
            new_df.loc[eliminator['Team1'].values[0], 'SeasonPosition'] = '4th Place'

    # Fill remaining places
    start_rank = 5
    nan_indexes = new_df[new_df['SeasonPosition'].isna()].index
    for i, idx in enumerate(nan_indexes, start=start_rank):
        new_df.at[idx, 'SeasonPosition'] = i

    return new_df


point_table_extension(2011)

C:\Users\MEGH BAVARVA\AppData\Local\Temp\ipykernel_19276\1616587337.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Winner' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_df.loc[final['Team1'].values[0], 'SeasonPosition'] = 'Winner'


,Match_played,Wins,No_result,Losses,Points,SeasonPosition
Team_name,,,,,,
Chennai Super Kings,16,11,0,5,22,Winner
Royal Challengers Bangalore,16,10,0,6,20,Runner Up
Mumbai Indians,16,10,0,6,20,3rd Place
Kolkata Knight Riders,15,8,0,7,16,5
Kings XI Punjab,14,7,0,7,14,6
Deccan Chargers,14,6,0,8,12,7
Rajasthan Royals,13,6,0,7,12,8
Kochi Tuskers Kerala,14,6,0,8,12,9
Delhi Capitals,14,4,1,9,9,10


In [ ]:
# ipl2.info()
def point_table_extension(season):
    season = str(season)
    new_df = point_table(season)

    final = ipl2[(ipl2['Season'].astype(str) == season) & (ipl2['MatchNumber'] == 'Final')]
    if not final.empty:
        if final['WinningTeam'].values[0] == final['Team1'].values[0]:
            new_df.loc[final['Team1'].values[0], 'SeasonPosition'] = 'Winner'
            new_df.loc[final['Team2'].values[0], 'SeasonPosition'] = 'Runner Up'
        else:
            new_df.loc[final['Team2'].values[0], 'SeasonPosition'] = 'Winner'
            new_df.loc[final['Team1'].values[0], 'SeasonPosition'] = 'Runner Up'

    qualifier = ipl2[(ipl2['Season'].astype(str) == season) & (ipl2['MatchNumber'] == 'Qualifier 2')]
    if not qualifier.empty:
        if qualifier['WinningTeam'].values[0] == qualifier['Team1'].values[0]:
            new_df.loc[qualifier['Team2'].values[0], 'SeasonPosition'] = '3rd Place'
        else:
            new_df.loc[qualifier['Team1'].values[0], 'SeasonPosition'] = '3rd Place'

    eliminator = ipl2[(ipl2['Season'].astype(str) == season) & (ipl2['MatchNumber'] == 'Eliminator')]
    if not eliminator.empty:
        if eliminator['WinningTeam'].values[0] == eliminator['Team1'].values[0]:
            new_df.loc[eliminator['Team2'].values[0], 'SeasonPosition'] = '4th Place'
        else:
            new_df.loc[eliminator['Team1'].values[0], 'SeasonPosition'] = '4th Place'

    # Fill remaining
    if 'SeasonPosition' not in new_df.columns:
        new_df['SeasonPosition'] = None

    start_rank = 5
    nan_indexes = new_df[new_df['SeasonPosition'].isna()].index
    for i, idx in enumerate(nan_indexes, start=start_rank):
        new_df.at[idx, 'SeasonPosition'] = f"{i}th Place"

    return new_df  # ✅ Return DataFrame

point_table_extension(2011)

,Match_played,Wins,No_result,Losses,Points,SeasonPosition
Team_name,,,,,,
Chennai Super Kings,16,11,0,5,22,Winner
Royal Challengers Bangalore,16,10,0,6,20,Runner Up
Mumbai Indians,16,10,0,6,20,3rd Place
Kolkata Knight Riders,15,8,0,7,16,5th Place
Kings XI Punjab,14,7,0,7,14,6th Place
Deccan Chargers,14,6,0,8,12,7th Place
Rajasthan Royals,13,6,0,7,12,8th Place
Kochi Tuskers Kerala,14,6,0,8,12,9th Place
Delhi Capitals,14,4,1,9,9,10th Place


In [10]:
balls=pd.read_csv("C:\\Users\\MEGH BAVARVA\\Desktop\\Data Scientist\\Pandas\\Task\\Task5\\IPL_Ball_by_Ball_2008_2022.csv")
matches=pd.read_csv("C:\\Users\\MEGH BAVARVA\\Desktop\\Data Scientist\\Pandas\\Task\\Task5\\IPL_Matches_2008_2022.csv")

balls.head(2)
matches.head(2)


season=balls.merge(matches[['ID','Season']],how='inner',on='ID')
season

season['batter'].unique()
season['bowler'].unique()

# season['IsBowlerWicket']=season['kind'].apply(lambda x: 1 if x in ('caught', 'caught and bowled', 'bowled', 'stumped','lbw', 'hit wicket') else 0)

# season['Bowler_run']=season['extra_type'].apply(lambda x: 0 if x in ('legbyes','byes') else 1) * season['total_run']

# season['LegalDilevery']=season['extra_type'].apply(lambda x: 0 if x in ('wides','noballs') else 1)

# season

# pcapdf=season.groupby(['Season','bowler']).agg({
#     'IsBowlerWicket':'sum',
#     'Bowler_run':'sum',
#     'LegalDilevery':'sum',
# })


# pcapdf['Bowler_Economy']=round((pcapdf['Bowler_run']/pcapdf['LegalDilevery'])*6,2)

# final=pcapdf.reset_index().sort_values(by=['IsBowlerWicket','Bowler_Economy'],ascending=[False,True]).drop_duplicates(subset=['Season'],keep='first').sort_values('Season').set_index('Season')
# final=final[['bowler','IsBowlerWicket','Bowler_Economy']].rename(columns={'bowler':'Best_Bowler','IsBowlerWicket':'Wickets','Bowler_Economy':'Economy'})
# final

array(['Mohammed Shami', 'Yash Dayal', 'LH Ferguson', 'Rashid Khan',
       'HH Pandya', 'R Sai Kishore', 'TA Boult', 'M Prasidh Krishna',
       'YS Chahal', 'OC McCoy', 'R Ashwin', 'Mohammed Siraj',
       'JR Hazlewood', 'GJ Maxwell', 'Shahbaz Ahmed', 'HV Patel',
       'PWH de Silva', 'Mohsin Khan', 'PVD Chameera', 'KH Pandya',
       'Avesh Khan', 'Ravi Bishnoi', 'AS Joseph', 'LS Livingstone',
       'Arshdeep Singh', 'K Rabada', 'NT Ellis', 'Harpreet Brar',
       'B Kumar', 'Washington Sundar', 'Fazalhaq Farooqi', 'J Suchith',
       'Umran Malik', 'R Shepherd', 'DR Sams', 'HR Shokeen', 'JJ Bumrah',
       'M Markande', 'RP Meredith', 'Ramandeep Singh', 'KK Ahmed',
       'A Nortje', 'SN Thakur', 'MR Marsh', 'Kuldeep Yadav', 'AR Patel',
       'Mukesh Choudhary', 'Simarjeet Singh', 'MJ Santner', 'M Pathirana',
       'MM Ali', 'PH Solanki', 'S Kaul', 'MK Lomror', 'UT Yadav',
       'TG Southee', 'SP Narine', 'CV Varun', 'AD Russell', 'N Rana',
       'JO Holder', 'K Gowtham', 'M

In [155]:
balls=pd.read_csv("C:\\Users\\MEGH BAVARVA\\Desktop\\Data Scientist\\Pandas\\Task\\Task5\\IPL_Ball_by_Ball_2008_2022.csv")
matches=pd.read_csv("C:\\Users\\MEGH BAVARVA\\Desktop\\Data Scientist\\Pandas\\Task\\Task5\\IPL_Matches_2008_2022.csv")

balls.head(2)
matches.head(2)


season=balls.merge(matches[['ID','Season']],how='inner',on='ID')
season['LegalDilevery']=season['extra_type'].apply(lambda x: 0 if x in ('wides','noballs') else 1)
season['Player_out_hai'] = ((season['isWicketDelivery'] == 1) & (season['batter'] == season['player_out'])).astype(int)


temp = season.groupby(['Season', 'batter']).agg({
    'batsman_run': 'sum',
    'LegalDilevery': 'sum',
    'Player_out_hai': 'sum'
}).reset_index()

# Compute average and strike rate
temp['Average'] = temp['batsman_run'] / temp['Player_out_hai']
temp['Strike_Rate'] = round((temp['batsman_run'] / temp['LegalDilevery']) * 100, 2)

# Handle infinite or undefined average (when Player_out_hai == 0)
temp['Average'] = temp.apply(
    lambda row: row['batsman_run'] if (row['Player_out_hai'] == 0) else round(row['Average'], 2),
    axis=1
)


temp.sort_values(by=['batsman_run','Strike_Rate','Average'], ascending=[False, False,False], inplace=True)
temp.drop_duplicates(subset=['Season'], keep='first', inplace=True)
final=temp.sort_values('Season').set_index('Season')[['batter','batsman_run','Strike_Rate','Average']].rename(columns={'batter':'Batsman','batsman_run':'Runs'})

final

# season.groupby(['Season','batter'])['batsman_run'].sum().reset_index().sort_values('batsman_run',ascending=False).drop_duplicates(subset=['Season'],keep='first').sort_values('Season').set_index('Season').rename(columns={'batter':'Best_Batsman','batsman_run':'Runs'})

,Batsman,Runs,Strike_Rate,Average
Season,,,,
2007/08,SE Marsh,616,140.00,68.44
2009,ML Hayden,572,145.55,57.20
2009/10,SR Tendulkar,618,133.48,47.54
2011,CH Gayle,608,183.69,67.56
2012,CH Gayle,733,162.53,61.08
2013,MEK Hussey,733,129.73,56.38
2014,RV Uthappa,660,138.08,44.00
2015,DA Warner,562,157.87,46.83
2016,V Kohli,973,152.75,81.08


In [165]:
def bowler_details(player_name):
    season['IsBowlerWicket']=season['kind'].apply(lambda x: 1 if x in ('caught', 'caught and bowled', 'bowled', 'stumped','lbw', 'hit wicket') else 0)

    season['Bowler_run']=season['extra_type'].apply(lambda x: 0 if x in ('legbyes','byes') else 1) * season['total_run']

    season['LegalDilevery']=season['extra_type'].apply(lambda x: 0 if x in ('wides','noballs') else 1)
    
    player = season[season['bowler'] == player_name]
    
    # Innings bowled
    innings_bowled = player.groupby(['Season'])['ID'].nunique()
    col1 = innings_bowled.reset_index(name="Innings")

    # Total wickets (exclude run-outs)
    total_wickets = player[player['IsBowlerWicket'] == 1].groupby(['Season'])['IsBowlerWicket'].sum()
    col2 = total_wickets.reset_index(name="Wickets")

    # Best spell (most wickets in a single match)
    best_spell = player[player['IsBowlerWicket'] == 1].groupby(['Season', 'ID'])['IsBowlerWicket'].sum().reset_index()
    best_spell = best_spell.sort_values(by=['Season', 'IsBowlerWicket'], ascending=[True, False])
    best_spell = best_spell.drop_duplicates(subset=['Season'], keep='first')
    col3 = best_spell[['Season', 'IsBowlerWicket']].rename(columns={'IsBowlerWicket': 'Best_spell'})

    # Total runs conceded
    runs_conceded = player.groupby(['Season'])['Bowler_run'].sum()
    col4 = runs_conceded.reset_index(name="Runs_conceded")

    # Legal deliveries
    legal_balls = player[player['LegalDilevery'] == 1].groupby(['Season'])['ballnumber'].count()
    col5 = legal_balls.reset_index(name="Balls_bowled")

    # Economy rate
    economy = (runs_conceded / legal_balls) * 6
    col6 = economy.reset_index(name="Economy")

    # Merge all
    stats = col1.merge(col2, on='Season', how='left') \
                .merge(col3, on='Season', how='left') \
                .merge(col4, on='Season', how='left') \
                .merge(col5, on='Season', how='left') \
                .merge(col6, on='Season', how='left')

    stats.fillna(0, inplace=True)

    # Final result
    return stats[['Season', 'Innings', 'Wickets', 'Best_spell', 'Runs_conceded', 'Economy']].to_dict(orient='records')


# bowler_details("Mohammed Shami")
season

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,...,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,Season,LegalDilevery,Player_out_hai,IsBowlerWicket,Bowler_run
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,...,0,NaN,NaN,NaN,Rajasthan Royals,2022,1,0,0,0
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,...,0,NaN,NaN,NaN,Rajasthan Royals,2022,1,0,0,0
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,...,0,NaN,NaN,NaN,Rajasthan Royals,2022,1,0,0,1
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,...,0,NaN,NaN,NaN,Rajasthan Royals,2022,1,0,0,0
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,...,0,NaN,NaN,NaN,Rajasthan Royals,2022,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225949,335982,2,14,5,P Kumar,I Sharma,SB Joshi,legbyes,0,1,...,0,NaN,NaN,NaN,Royal Challengers Bangalore,2007/08,1,0,0,0
225950,335982,2,14,6,SB Joshi,I Sharma,P Kumar,NaN,1,0,...,0,NaN,NaN,NaN,Royal Challengers Bangalore,2007/08,1,0,0,1
225951,335982,2,14,7,P Kumar,I Sharma,SB Joshi,NaN,0,0,...,0,NaN,NaN,NaN,Royal Challengers Bangalore,2007/08,1,0,0,0
225952,335982,2,15,1,SB Joshi,LR Shukla,P Kumar,wides,0,1,...,0,NaN,NaN,NaN,Royal Challengers Bangalore,2007/08,0,0,0,1


In [156]:
# Merge ball-level and match-level data
merged_df = balls.merge(matches, how='inner', on='ID')

# Get Player of the Match per match
player_of_match_df = merged_df.groupby(['Season', 'ID'])['Player_of_Match'].first().reset_index()

# Filter rows where the Player of the Match was the batter
pom_batting = merged_df[merged_df['batter'] == merged_df['Player_of_Match']]

# Batting runs and balls faced
batting_runs = pom_batting.groupby(['Season', 'ID'])['batsman_run'].sum().reset_index()
balls_faced = pom_batting.groupby(['Season', 'ID'])['ballnumber'].count().reset_index()

# Calculate bowler-run (ignore legbyes and byes)
merged_df['bowler_run'] = merged_df['extra_type'].apply(lambda x: 0 if x in ('legbyes', 'byes') else 1)

# Identify wickets credited to bowler
merged_df['is_bowler_wicket'] = merged_df['kind'].apply(
    lambda x: 1 if x in ('caught', 'caught and bowled', 'bowled', 'stumped', 'lbw', 'hit wicket') else 0)

# Filter rows where Player of the Match was the bowler
pom_bowling = merged_df[merged_df['bowler'] == merged_df['Player_of_Match']]

# Wickets taken
bowling_wickets = pom_bowling.groupby(['Season', 'ID'])['is_bowler_wicket'].sum().reset_index()

# Runs conceded by bowler
bowling_runs = pom_bowling[pom_bowling['bowler_run'] == 1]
runs_conceded = bowling_runs.groupby(['Season', 'ID'])['total_run'].sum().reset_index()

# Merge all data
final_df = player_of_match_df \
    .merge(batting_runs, how='outer', on=['Season', 'ID']) \
    .merge(balls_faced, how='outer', on=['Season', 'ID']) \
    .merge(bowling_wickets, how='outer', on=['Season', 'ID']) \
    .merge(runs_conceded, how='outer', on=['Season', 'ID'])

# Create Batting and Bowling Figure strings (e.g., 45/30 or 3/25)
final_df["Batting_Figure"] = final_df['batsman_run'].astype('Int32').astype(str) + "/" + final_df['ballnumber'].astype('Int32').astype(str)
final_df["Bowling_Figure"] = final_df['is_bowler_wicket'].astype('Int32').astype(str) + "/" + final_df['total_run'].astype('Int32').astype(str)

# Final Display
final_df[['Season','ID', 'Player_of_Match', 'Batting_Figure', 'Bowling_Figure']].set_index('Season')

,ID,Player_of_Match,Batting_Figure,Bowling_Figure
Season,,,,
2007/08,335982,BB McCullum,158/77,<NA>/<NA>
2007/08,335983,MEK Hussey,116/54,<NA>/<NA>
2007/08,335984,MF Maharoof,<NA>/<NA>,2/11
2007/08,335985,MV Boucher,39/20,<NA>/<NA>
2007/08,335986,DJ Hussey,38/45,1/35
...,...,...,...,...
2022,1304116,Harpreet Brar,<NA>/<NA>,3/26
2022,1312197,DA Miller,68/38,<NA>/<NA>
2022,1312198,RM Patidar,112/55,<NA>/<NA>


In [ ]:
def details(player_name):        
    player=season[season['batter']==player_name]
    player

    innings_played=player.groupby(['Season'])['ID'].nunique()
    col1=innings_played.reset_index(name="innings_played")
    col1

    batsman_runs=player.groupby(['Season'])['total_run'].sum()
    col2=batsman_runs.reset_index(name="Total_runs")
    col2

    batsman_out = season[season['player_out'] == player_name] # nonstrike uper hoy and runout thaye to pun out ave ne 
    req= batsman_out.groupby('Season')['isWicketDelivery'].sum()
    col3= req.reset_index(name="total_no_out")

    Average=batsman_runs/req
    col4=Average.reset_index(name="average")

    highest_runs=player.groupby(['Season','ID'])['batsman_run'].sum().reset_index().sort_values(by=['batsman_run'],ascending=False).drop_duplicates(subset=['Season'],keep='first').set_index('Season').sort_index()
    col5=highest_runs.reset_index()[['Season','batsman_run']]

    balls_not_wide=player[~(player["extra_type"]=='wides')]
    balls_not_wide
    batsman_balls=balls_not_wide.groupby(['Season'])['ballnumber'].count()
    col6=batsman_balls.reset_index(name="Ball_played")

    strike_rate=round((batsman_runs/batsman_balls)*100,2)
    col7=strike_rate.reset_index(name='StrikeRate')
    col7
        
    stats=col1.merge(col2,on='Season').merge(col3,on='Season').merge(col4,on='Season').merge(col5,on='Season').merge(col6,on='Season').merge(col7,on='Season')

    stats.rename(columns={'innings_played':"Innings",
                        'average':'Average',
                        'batsman_run':'Highest_runs',
                        'StrikeRate':'Strike_rate'},inplace=True)
    return stats[['Season','Innings','Total_runs','Average','Highest_runs','Strike_rate']]

# player

details("MS Dhoni")

,Season,Innings,Total_runs,Average,Highest_runs,Strike_rate
0,2007/08,14,433,43.300000,65,139.68
1,2009,13,343,42.875000,58,131.42
2,2009/10,11,300,33.333333,66,142.86
3,2011,13,397,44.111111,70,160.73
4,2012,17,371,30.916667,51,133.45
5,2013,16,482,43.818182,67,170.32
6,2014,15,402,80.400000,57,160.80
7,2015,17,390,32.500000,53,127.87
8,2016,12,303,43.285714,64,144.29
9,2017,15,307,27.909091,61,122.80


In [33]:
ipl

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,N,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,N,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,N,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,N,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,335986,Kolkata,2008-04-20,2007/08,4,Kolkata Knight Riders,Deccan Chargers,Eden Gardens,Deccan Chargers,bat,N,Kolkata Knight Riders,Wickets,5.0,NaN,DJ Hussey,"['WP Saha', 'BB McCullum', 'RT Ponting', 'SC G...","['AC Gilchrist', 'Y Venugopal Rao', 'VVS Laxma...",BF Bowden,K Hariharan
946,335985,Mumbai,2008-04-20,2007/08,5,Mumbai Indians,Royal Challengers Bangalore,Wankhede Stadium,Mumbai Indians,bat,N,Royal Challengers Bangalore,Wickets,5.0,NaN,MV Boucher,"['L Ronchi', 'ST Jayasuriya', 'DJ Thornely', '...","['S Chanderpaul', 'R Dravid', 'LRPL Taylor', '...",SJ Davis,DJ Harper
947,335984,Delhi,2008-04-19,2007/08,3,Delhi Daredevils,Rajasthan Royals,Feroz Shah Kotla,Rajasthan Royals,bat,N,Delhi Daredevils,Wickets,9.0,NaN,MF Maharoof,"['G Gambhir', 'V Sehwag', 'S Dhawan', 'MK Tiwa...","['T Kohli', 'YK Pathan', 'SR Watson', 'M Kaif'...",Aleem Dar,GA Pratapkumar
948,335983,Chandigarh,2008-04-19,2007/08,2,Kings XI Punjab,Chennai Super Kings,"Punjab Cricket Association Stadium, Mohali",Chennai Super Kings,bat,N,Chennai Super Kings,Runs,33.0,NaN,MEK Hussey,"['K Goel', 'JR Hopes', 'KC Sangakkara', 'Yuvra...","['PA Patel', 'ML Hayden', 'MEK Hussey', 'MS Dh...",MR Benson,SL Shastri


In [55]:
def point_table(season):
    df = ipl2[ipl2['Season'].astype(str) == str(season)]
    teams = np.union1d(df['Team1'].unique(), df['Team2'].unique())

    new_dataframe = pd.DataFrame()
    new_dataframe['Team_name'] = teams

    new_dataframe['Match_played'] = new_dataframe['Team_name'].apply(lambda x: matched_played(df, x))
    new_dataframe['Wins'] = new_dataframe['Team_name'].apply(lambda x: match_won(df, x))
    new_dataframe['No_result'] = new_dataframe['Team_name'].apply(lambda x: no_result(df, x))
    new_dataframe['Losses'] = new_dataframe['Match_played'] - new_dataframe['Wins'] - new_dataframe['No_result']
    new_dataframe['Points'] = new_dataframe['Wins'] * 2 + new_dataframe['No_result']

    new_dataframe.sort_values('Points', ascending=False, inplace=True)
    new_dataframe.set_index('Team_name', inplace=True)

    return new_dataframe  # ✅ return DataFrame instead of dict


point_table(2016)

,Match_played,Wins,No_result,Losses,Points
Team_name,,,,,
Sunrisers Hyderabad,17,11,0,6,22
Gujarat Lions,16,9,0,7,18
Royal Challengers Bangalore,16,9,0,7,18
Kolkata Knight Riders,15,8,0,7,16
Mumbai Indians,14,7,0,7,14
Delhi Capitals,14,7,0,7,14
Rising Pune Supergiants,14,5,0,9,10
Kings XI Punjab,14,4,0,10,8


In [ ]:
# code here

# def point_table_extension(season):
# new_df = point_table(season)  # should return a DataFrame

# final = ipl[(ipl['Season'] == season) & (ipl['MatchNumber'] == 'Final')]
# if final['WinningTeam'].values[0] == final['Team1'].values[0]:
#     new_df.loc[final['Team1'].values[0], 'SeasonPosition'] = 'Winner'
#     new_df.loc[final['Team2'].values[0], 'SeasonPosition'] = 'Runner Up'
# else:
#     new_df.loc[final['Team2'].values[0], 'SeasonPosition'] = 'Winner'
#     new_df.loc[final['Team1'].values[0], 'SeasonPosition'] = 'Runner Up'

# qualifier = ipl[(ipl['Season'] == season) & (ipl['MatchNumber'] == 'Qualifier 2')]
# if qualifier['WinningTeam'].values[0] == qualifier['Team1'].values[0]:
#     new_df.loc[qualifier['Team2'].values[0], 'SeasonPosition'] = '3rd Place'
# else:
#     new_df.loc[qualifier['Team1'].values[0], 'SeasonPosition'] = '3rd Place'

# eliminator = ipl[(ipl['Season'] == season) & (ipl['MatchNumber'] == 'Eliminator')]
# if eliminator['WinningTeam'].values[0] == eliminator['Team1'].values[0]:
#     new_df.loc[eliminator['Team2'].values[0], 'SeasonPosition'] = '4th Place'
# else:
#     new_df.loc[eliminator['Team1'].values[0], 'SeasonPosition'] = '4th Place'

# # Start ranking from 5th place for NaN values
# # start_rank = 5

# # Ensure 'SeasonPosition' column exists
# # if 'SeasonPosition' not in new_df.columns:
# #     new_df['SeasonPosition'] = np.nan

# # Get indexes where SeasonPosition is NaN
# nan_indexes = new_df[new_df['SeasonPosition'].isna()].index

# # Assign "5th Place", "6th Place", ... to NaNs in order
# for i, idx in enumerate(nan_indexes, start=start_rank):
#     new_df.at[idx, 'SeasonPosition'] = f'{i}th Place'



# return new_df


# point_table_extension(2016)

NameError: name 'season' is not defined

In [ ]:
# new_df = point_table(2016)

# final = ipl2[(ipl2['Season'] == 2016) & (ipl2['MatchNumber'] == 'Final')]
# if not final.empty:
#     if final['WinningTeam'].values[0] == final['Team1'].values[0]:
#         new_df.loc[final['Team1'].values[0], 'SeasonPosition'] = 'Winner'
#         new_df.loc[final['Team2'].values[0], 'SeasonPosition'] = 'Runner Up'
#     else:
#         new_df.loc[final['Team2'].values[0], 'SeasonPosition'] = 'Winner'
#         new_df.loc[final['Team1'].values[0], 'SeasonPosition'] = 'Runner Up'
    
# new_df


,Match_played,Wins,No_result,Losses,Points
Team_name,,,,,
Sunrisers Hyderabad,17,11,0,6,22
Gujarat Lions,16,9,0,7,18
Royal Challengers Bangalore,16,9,0,7,18
Kolkata Knight Riders,15,8,0,7,16
Mumbai Indians,14,7,0,7,14
Delhi Capitals,14,7,0,7,14
Rising Pune Supergiants,14,5,0,9,10
Kings XI Punjab,14,4,0,10,8


In [66]:

def point_table_extension(val):

    new_df = point_table(val)
    
    season_val = str(val)

    final = ipl[(ipl['Season'] == season_val) & (ipl['MatchNumber'] == 'Final')]
    if final['WinningTeam'].values[0] == final['Team1'].values[0]:
        new_df.loc[final['Team1'].values[0], 'SeasonPosition'] = 'Winner'
        new_df.loc[final['Team2'].values[0], 'SeasonPosition'] = 'Runner Up'
    else:
        new_df.loc[final['Team2'].values[0], 'SeasonPosition'] = 'Winner'
        new_df.loc[final['Team1'].values[0], 'SeasonPosition'] = 'Runner Up'

    qualifier = ipl[(ipl['Season'] == season_val) & (ipl['MatchNumber'] == 'Qualifier 2')]
    if qualifier['WinningTeam'].values[0] == qualifier['Team1'].values[0]:
        new_df.loc[qualifier['Team2'].values[0], 'SeasonPosition'] = '3rd Place'
    else:
        new_df.loc[qualifier['Team1'].values[0], 'SeasonPosition'] = '3rd Place'

    eliminator = ipl[(ipl['Season'] == season_val) & (ipl['MatchNumber'] == 'Eliminator')]
    if eliminator['WinningTeam'].values[0] == eliminator['Team1'].values[0]:
        new_df.loc[eliminator['Team2'].values[0], 'SeasonPosition'] = '4th Place'
    else:
        new_df.loc[eliminator['Team1'].values[0], 'SeasonPosition'] = '4th Place'


    start_rank = 5

    # Ensure 'SeasonPosition' column exists
    if 'SeasonPosition' not in new_df.columns:
        new_df['SeasonPosition'] = np.nan

    # Get indexes where SeasonPosition is NaN
    nan_indexes = new_df[new_df['SeasonPosition'].isna()].index

    # Assign "5th Place", "6th Place", ... to NaNs in order
    for i, idx in enumerate(nan_indexes, start=start_rank):
        new_df.at[idx, 'SeasonPosition'] = i

    return new_df


point_table_extension(2022)
# final
# ipl[(ipl['Season'] == '2022')]
    


,Match_played,Wins,No_result,Losses,Points,SeasonPosition
Team_name,,,,,,
Gujarat Titans,16,12,0,4,24,Winner
Rajasthan Royals,17,10,0,7,20,Runner Up
Royal Challengers Bangalore,16,9,0,7,18,3rd Place
Lucknow Super Giants,15,9,0,6,18,4th Place
Punjab Kings,14,7,0,7,14,5
Delhi Capitals,14,7,0,7,14,6
Sunrisers Hyderabad,14,6,0,8,12,7
Kolkata Knight Riders,14,6,0,8,12,8
Chennai Super Kings,14,4,0,10,8,9


In [63]:
ipl['Season']

0         2022
1         2022
2         2022
3         2022
4         2022
        ...   
945    2007/08
946    2007/08
947    2007/08
948    2007/08
949    2007/08
Name: Season, Length: 950, dtype: object

In [1]:
import numpy as np
import pandas as pd

ipl = pd.read_csv("C:\\Users\\MEGH BAVARVA\\Desktop\\Data Scientist\\Project\\API_development\\Code\\IPL_Matches_2008_2022.csv")
print(ipl.columns)

# team1 = ipl['Team1']
# team1
# team2 = ipl['Team2'].unique()

# # all_teams = pd.unique(np.concatenate((team1, team2)))
# all_teams

Index(['ID', 'City', 'Date', 'Season', 'MatchNumber', 'Team1', 'Team2',
       'Venue', 'TossWinner', 'TossDecision', 'SuperOver', 'WinningTeam',
       'WonBy', 'Margin', 'method', 'Player_of_Match', 'Team1Players',
       'Team2Players', 'Umpire1', 'Umpire2'],
      dtype='object')
